# はじめに
本ハンズオンでは機械学習モデルの解釈性をテーマに、どの特徴量がどの程度の影響があるかについて解析する手法について Amazon SageMaker の組み込みアルゴリズム XGBoost を用いて実践します。

## 本ハンズオンで学べる内容
- SageMaker の組み込みアルゴリズムである XGBoost を用いた機械学習モデルの学習
- SageMaker を用いた際の XGBoost での特徴量重要度の求め方
- SageMaker でのバッチ推論の実施方法
- Partianl Dependency Plot(PDP) による特徴量の変化による目的変数への影響の可視化

In [ ]:
# Define IAM role
import time
import pickle as pkl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer
from sagemaker.session import s3_input

sess = sagemaker.Session()
role = get_execution_role()

## データの準備


今回は [UCI Machine Learning Repogitory](https://archive.ics.uci.edu/ml/index.php) にある "Wine Quality" というデータセットを活用します。

それぞれのワインがアルコール度数や、pH 、残糖量といった計測指標と共に、その品質が 3〜8 で段階的に評価されているデータセットです。

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

In [ ]:
data = pd.read_csv("winequality-red.csv",sep=";",encoding="utf-8")
data.head()

各カラムは下記を意味しています。
- alcohol: アルコール度数
- cholorides: 塩化ナトリウム濃度
- citric acid: クエン酸濃度
- density: 密度
- fixed acidity: 酸性度 
- pH: pH
- residual sugar: 残糖含有量
- sulphates: 硫化カリウム含有量
- total sulfur dioxide: 総二酸化硫黄含有量
- free sulfur dioxide: 遊離二酸化硫黄含有量
- volatile acidity: 揮発性酸度
- quality: 品質

ヒストグラムを描いて、データの分布を確認しましょう。

In [ ]:
# 特徴量をヒストグラムとして可視化
hist = data.hist(bins=30, sharey=True, figsize=(15, 15))

## 相関行列を求める
データを確認する意味で、それぞれの特徴量同士の相関を見てみましょう。`fixed acidity` と `citric acid` や `density` はやや強い相関が見える、`quality` と強い相関のある特徴量があるとは言えない、などの傾向がつかめます。

In [ ]:
# 相関行列を求める
display(data.corr())
pd.plotting.scatter_matrix(data, figsize=(12, 12))
plt.show()

## データの分割
データを学習用、検証用、テスト用データへ分割します。学習＆検証用データで構築したモデルに対してテストデータで推論と、その予測値に対してどの特徴量がどの程度寄与しているか、または変化した際に、どの程度影響を及ぼすのかを解析します。

In [ ]:
# SageMakerの組み込みアルゴリズムへの対応のためデータの並び替え
model_data = pd.concat([data['quality'], data.drop(['quality'], axis=1)], axis=1)

# 学習用データ、検証用データ、テスト用データへ分割
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])

# 学習用データ、検証用データの保存
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

# アップロード先を指定
bucket = sess.default_bucket()
prefix = 'sagemaker/wine-quality-interpretability' 

# csvファイルとして保存された学習用データ、検証用データをS3バケット上にアップロード
sagemaker_session = sagemaker.Session()
input_train = sagemaker_session.upload_data(path='train.csv', key_prefix=prefix)
input_validation = sagemaker_session.upload_data(path='validation.csv', key_prefix=prefix)

s3_input_train = s3_input(s3_data=input_train, content_type='text/csv')
s3_input_validation = s3_input(s3_data=input_validation, content_type='text/csv')

## 学習
今回は SageMaker の組み込みアルゴリズムの中にある XGBoost を使います。まず、XGBoost のコンテナの場所を取得します。コンテナ自体は SageMaker 側で用意されているのでそれを指定します。学習のためにハイパーパラメータや、学習のインスタンスの数やタイプを指定して学習を開始します。XGBoostのhyperparameterに関する詳細は [github](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst) をご確認下さい。

In [ ]:
container = get_image_uri(boto3.Session().region_name, 'xgboost')

# 学習ジョブの名前を指定して下さい。XXXXは適宜変更して下さい。
job_name="xgb-wine-interpretability-XXXX"

# どのような学習インスタンスを使うのかの設定
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    sagemaker_session=sess)

# XGBoostアルゴリズムのハイパラ設定
xgb.set_hyperparameters(max_depth=5,
                        alpha=0.05,
                        eta=0.2,
                        min_child_weight=2,
                        subsample=0.8,
                        objective='reg:linear',
                        num_round=100)

# 学習の開始
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}, job_name=job_name)

## 学習させたモデルから特徴量重要度を取得

SageMaker で学習されたモデルは s3 のバケットに`model.tar.gz`として保存されています。
XGBoost では、どの特徴量がモデルの予測精度の向上に役立ったかは学習時に計算され、`xgboost-model`の中に保存されており、`get_score()` メソッドで呼び出せます。

ノートブックインスタンス上に XGBoost のライブラリをインストールすることで、学習済モデルを活用することができます。

In [ ]:
!conda install -c conda-forge -y xgboost

s3 のバケットから学習済みのモデルをダウンロードし、特徴量重要度をプロットして可視化してみましょう。

In [ ]:
# 学習させたxgboostモデルのダウンロード
s3 = boto3.client('s3')
s3.download_file(Bucket=bucket, Key= job_name + '/output/model.tar.gz', Filename = 'model.tar.gz')

# xgboostモデルの解凍
!tar -zxvf model.tar.gz

# ダウンロードしてきたxgboostモデルの読み込み
xgb_model = pkl.load(open("xgboost-model", 'rb'))

# 特徴量重要度を読み込むためのデータフレームの準備
dict_varImp = xgb_model.get_score(importance_type = 'weight')
df_ = pd.DataFrame(dict_varImp, index = ['varImp']).transpose().reset_index()
df_.columns = ['feature', 'fscore']

# 上位10個の特徴量重要度を描写
df_['fscore'] = df_['fscore'] / df_['fscore'].max()
df_.sort_values('fscore', ascending = False, inplace = True)
df_ = df_[0:11]
df_.sort_values('fscore', ascending = True, inplace = True)

fscore = df_['fscore']
feature = df_['feature']

# 特徴量重要度が特徴量の名前を保持していないため、データセットのカラム名からマッピング
mapper = {'f{0}'.format(i): v for i, v in enumerate(model_data.columns.drop('quality'))}
mapped_feature = [mapper[f] for f in df_["feature"]]

plt.figure(figsize=(10,10))
plt.title('XGBoost Feature Importance Top10', fontsize = 15)
plt.yticks(fontsize=15)
plt.barh(mapped_feature,fscore)

## Partial Dependency Plotの活用
Partial Dependency Plotは特徴量の寄与度だけでなく、その特徴量の変化した際にどの程度目的変数が変化するかを確認するために活用されます。 手順は下記になります。

- 検証データのある点を元にして、影響を見たい1つの特徴量だけを変化させたデータセットを作成する
- そのデータに対して学習済モデルで推論をを実施する
- 1つの特徴量が変化するとそれぞれのデータ点の予測はどう変わるのかを計算する
- 各サンプルの変化や、その平均をプロットする

今回は特徴量重要度が高かった `volatile acidity` に対して 0.2 〜 1.0 の間で変化させたデータセットを作成し、それぞれにおいて推論することで Partial Dependency Plotを描いていきます。

In [ ]:
# 対象となる特徴量を設定      
target_column = 'volatile acidity'
target_range = [round(0.2 + 0.05*x, 2) for x in range(0, 17, 1)]

# アップロード先を指定
bucket = sess.default_bucket()
prefix = 'sagemaker/wine-quality-interpretability'

input_data_list = []
for v in target_range:
    print("各サンプルで、{} が {} だった場合のデータセットを作成し s3 へアップロード".format(target_column, v))
    
    # 全てのサンプルデータの`alcohol`の値が v になったデータセットを作成し保存
    pdp_df = data.copy().drop(['quality'], axis=1)
    pdp_df[target_column] = v    
    pdp_channel = prefix + '/test_pdp_{}_{}'.format(target_column, v)       
    pdp_df.to_csv("test_X_{}_{}.csv".format(target_column, v), header=False, index=False)
    
    # csvファイルとして保存された学習用データ、検証用データをS3バケット上にアップロード
    input_data = sagemaker_session.upload_data(path="test_X_{}_{}.csv".format(target_column, v), bucket=bucket, key_prefix=pdp_channel)
    
    # アップロードされたデータ
    # s3_input_test = s3_input(s3_data=input_data, content_type='text/csv')

    # アップロード先のパス
    s3_pdp_data = 's3://{}/{}'.format(bucket, pdp_channel)
    s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
    
    input_data_list.append(input_data)

## SageMaker のバッチ変換ジョブを用いた推論


In [ ]:
# バッチ変換用のインスタンスを設定
xgb_transformer = xgb.transformer(instance_count=1,
                                  instance_type='ml.m4.xlarge',
                                  strategy='MultiRecord',
                                  assemble_with='Line',
                                  output_path=s3_output_location)

job_target_column = target_column.replace(' ', '-')

for i in range(len(target_range)):
    # バッチ変換用ジョブの実行
    xgb_transformer.transform(data=input_data_list[i],
                              job_name =  "transform-{}-{}".format(job_target_column, i),
                              content_type='text/csv',
                              split_type='Line')

それぞれのバッチ変換ジョブの状態を boto3 の SageMaker クライアントで `describe_transform_job()` を実行することで確認し、全てのジョブが終わるまで待ちます。

In [ ]:
client = boto3.client('sagemaker')

while True:
    response_list = []
    
    for i in range(len(target_range)):
        TransformJobName = "transform-{}-{}".format(job_target_column, i)
        response = client.describe_transform_job(TransformJobName=TransformJobName)
        response_list.append(response['TransformJobStatus'])
    
    if all(response == 'Completed' for response in response_list):
        print("Transform finished")
        break
        
    elif any(response == 'Failed' for response in response_list):
        print("Transform Failed")
        break
        
    else:
        print('Transform InProgress')
        time.sleep(30)

`volatile acidity` の値を変化させたそれぞれのサンプルにおいて品質がどのように変化するか、またそのデータセット全体における平均はどのように変化するか見てみましょう。

In [ ]:
# バッチ変換データのダウンロード
s3 = boto3.client('s3')
df_plot = pd.DataFrame()

for i in range(len(target_range)):
    s3.download_file(Bucket=bucket,
                     Key= prefix + "/output/" + "test_X_{}_{}.csv.out".format(target_column, target_range[i]),
                     Filename = "test_X_{}_{}.csv.out".format(job_target_column, target_range[i]))
    
    df_tmp =  pd.read_csv("test_X_{}_{}.csv.out".format(job_target_column, target_range[i]), names=[str(target_range[i])])
    df_plot = pd.concat([df_plot, df_tmp], axis=1)

    
# プロットの実行
plt.figure(figsize=(10,10))
plt.title('Partial Dependency Plot: volatile acidity', fontsize = 15)
plt.yticks(fontsize=15)
plt.xlabel('volatile acidity', fontsize = 15)
plt.ylabel('quality', fontsize = 15)
plt.plot(df_plot.T.index,df_plot.T, linewidth=0.2, color='blue')
plt.plot(df_plot.mean().index,df_plot.mean(), linewidth=5, color='red')

このプロットからは `volatile acidity` の上昇が品質の悪化に緩やかに影響し、特に 0.9 以上になると品質の劣化が著しくなる傾向が見えます。また、それぞれのサンプルでその影響具合が異ることも見てとれます。 